In [1]:
# _*_ coding: utf-8 _*_
import re
import nltk
import os
from nltk.tokenize import RegexpTokenizer

nltk.download('punkt')

import import_ipynb
from CommonModule.Handle_Dir import mkdir_p, del_folder
from CommonModule.ArticleHandler import Article, ArticleReader

importing Jupyter notebook from C:\Users\aqaq9\Documents\Article-Summarizer\articleSummary-Jupyter\Text-Preprocessing\CommonModule\Handle_Dir.ipynb
importing Jupyter notebook from C:\Users\aqaq9\Documents\Article-Summarizer\articleSummary-Jupyter\Text-Preprocessing\CommonModule\ArticleHandler.ipynb


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aqaq9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
BASE_DIR = "/data/TestDir/sample_articles"
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.retokenize = RegexpTokenizer("[\w]+")
        self.swords = []
        self.tokenizer = {}
        self.tagger = Komoran()

    def removeDuplicateSpace(self, text):
        return re.sub('\s+', ' ', text)  # 중복 공백, 탭, 개행 제거
    
    def removeSpecialChar(self, text):
        return ' '.join(self.retokenize.tokenize(text))


    def loadSwords(self, filename):
        self.swords = []
        with open(filename, 'r', encoding='utf-8') as f:
            swords = f.readlines()
            self.swords = [sword.strip() for sword in self.swords]

        self.tokenizer = lambda sent: filter(lambda x:x not in self.swords, sent.split())

        return self.swords
        
    def removeSwords(self, text):
        return ' '.join([token for token in list(self.tokenizer(text))]) 

In [7]:
def saveTextFile(baseDir, media, filename, sentences):

    mkdir_p(os.path.join(baseDir, media))
    save_path = os.path.join(os.path.join(baseDir, media), filename)

    with open(save_path, 'w', encoding='utf-8') as f:
        f.write('/n'.join([sentence for sentence in sentences if sentence is not '']))


In [ ]:
del_folder(PREPROCESSED_PATH)
del_folder(PRETTY_PATH)

preprocessor = TextPreprocessor()
preprocessor.loadSwords(SWORDS_PATH)

if __name__ == '__main__':
    
    media_list = sorted(os.listdir(ORIGIN_PATH))

    for media_name in media_list:

        media_path = os.path.join(ORIGIN_PATH, media_name)
        article_list = sorted(os.listdir(media_path))

        for article_name in article_list:
            reader = ArticleReader(os.path.join(media_path, article_name))

            article = Article(list(filter(None, reader)))
            
            if len(article.content) is 0 : continue # 본문이 없는 경우, 학습 데이터에서 제외함
            
            prettyLine = []
            preprocessedLine = []
            for line in article.readContent() :
                cleanLine = preprocessor.removeDuplicateSpace(line)
                cleanLine = preprocessor.removeSpecialChar(cleanLine)
                
                rmSwordLine = preprocessor.removeSwords(cleanLine)
                if rmSwordLine is '': continue
                    
                preprocessedLine.append(rmSwordLine)

            saveTextFile(PREPROCESSED_PATH, media_name, article_name, preprocessedLine)
            saveTextFile(PRETTY_PATH, media_name, article_name, article.readContent())